In [ ]:
#downloading the required packages
%pip install pandas
%pip install pandas_ta
%pip install numpy
%pip install matplotlib
%pip install statsmodels
%pip install pandas_datareader
%pip install datetime
%pip install yfinance
%pip install sklearn
%pip install PyPortfolioOpt

In [ ]:
# from statsmodels.regression.rolling import RollingOLS
# import pandas_datareader.data as web
# import matplotlib.pyplot as plt
# import statsmodels.api as sm
# import pandas as pd
# import numpy as np
# import yfinance as yf
# import pandas_ta
# import warnings
# warnings.filterwarnings('ignore')


# sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

# sp500['Symbol'] = sp500['Symbol'].str.replace('.', '-')

# symbols_list = sp500['Symbol'].unique().tolist()

# symbols_list

# end_date = '2025-01-21'

# start_date = pd.to_datetime(end_date) - pd.DateOffset(365*10)

# start_date

# df = yf.download(tickers = symbols_list, start= start_date, end = end_date).stack()

df

[*********************100%***********************]  503 of 503 completed


Price                   Close        High         Low        Open       Volume
Date       Ticker                                                             
2015-01-26 A        35.902466   35.920804   35.278872   35.572328    1510900.0
           AAPL     25.097639   25.377242   25.031068   25.239659  222460000.0
           ABBV     40.729492   40.865624   39.984005   40.359988    8181100.0
           ABT      36.159973   36.200924   35.562082   35.824173    4376200.0
           ACGL     18.932390   18.935561   18.659799   18.862657    1662000.0
...                       ...         ...         ...         ...          ...
2025-01-17 XYZ      86.959999   88.150002   86.099998   87.000000    5797000.0
           YUM     124.121735  126.132323  123.992980  125.696536    1604900.0
           ZBH     109.042999  109.451068  107.958146  108.475689    1097100.0
           ZBRA    405.709991  407.290009  402.290009  406.040009     270600.0
           ZTS     164.284439  168.027853  164.205212  167.265301    4657200.0

[1226482 rows x 5 columns]